In [19]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
import gmaps
from api_keys import g_key
gmaps.configure(api_key = g_key)




In [22]:
output_data_file = "Output_CSV/weather_data.csv"
types_df = pd.read_csv(output_data_file)
weather_data_df = types_df.drop(columns = ["Unnamed: 0"])
weather_data_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Butaritari,99,KI,1595191634,75,3.07,172.79,83.44,17.36
1,Tegul'det,73,RU,1595191634,92,57.31,88.17,56.75,5.46
2,Rikitea,38,PF,1595191634,73,-23.12,-134.97,68.38,16.20
3,Bredasdorp,0,ZA,1595191634,93,-34.53,20.04,55.40,1.07
4,Khatanga,96,RU,1595191635,79,71.97,102.50,51.67,4.59
...,...,...,...,...,...,...,...,...,...
554,Brainerd,40,US,1595191574,39,46.36,-94.20,80.60,17.22
555,San Dionisio del Mar,0,MX,1595191697,58,16.33,-94.75,91.49,16.35
556,Otofuke,33,JP,1595191697,93,42.99,143.20,64.69,1.16
557,San Jose,75,US,1595191625,50,37.34,-121.89,82.00,6.93


In [23]:
gmaps.configure(api_key = g_key)

In [29]:
locations = weather_data_df[["Lat", "Lng"]]

humidity = weather_data_df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
newtypes_df = pd.DataFrame(weather_data_df, columns = ["City","Max Temp","Wind Speed", "Cloudiness"])

max_temp = (newtypes_df["Max Temp"] <= 80) & (newtypes_df["Max Temp"] > 70)
wind_speed = newtypes_df["Wind Speed"] < 10
cloudiness = newtypes_df["Cloudiness"] == 0

newtypes_df = [max_temp & wind_speed & cloudiness]

In [43]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country" "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,CountryLat,Lng,Hotel Name
0,Butaritari,NaN,172.79,
1,Tegul'det,NaN,88.17,
2,Rikitea,NaN,-134.97,
3,Bredasdorp,NaN,20.04,
4,Khatanga,NaN,102.50,
...,...,...,...,...
554,Brainerd,NaN,-94.20,
555,San Dionisio del Mar,NaN,-94.75,
556,Otofuke,NaN,143.20,
557,San Jose,NaN,-121.89,


In [ ]:
target_coordinates = f"{hotel_df['CountryLat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)



In [ ]:
pprint(response.json(), depth=1)

In [ ]:
hotel_df.iloc[0][0]

In [ ]:
hotel_name = []
params = {
    "radius": 5000,
    "types": "hotel"   
}

for index, row in hotel_df.iterrows():
    
    lat = row["CountryLat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params = params)
    
    name_address = name_address.json()
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except: 
            hotel_name.append(np.nan)
hotel_name